In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
from pandas.api.types import CategoricalDtype
import glob

sns.set_theme(style="ticks", color_codes=True)
sns.set(style="ticks")

def LoadData(base_dir):
    search_string="/*.csv"
    dfs = []
    for filename in glob.glob(base_dir+search_string, recursive=True):
        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0, skipinitialspace=True)
        dfs.append(df)
        df = pd.concat(dfs, axis=0, ignore_index=True)
    return df
    
###### LOAD DATA ############
#baseline='/home/daniel/rosbag/CFEAR_EVAL/eval_storage/8_grid_search_2022-04-04_2143'
baseline='./data/'


df=LoadData(baseline)

df = df.rename(columns={'resolution r': 'resolution r [m]'})
df = df.rename(columns={'Trans.err.(%)': 'Transl. Error[%]'})
df = df.rename(columns={'Rot.err.(deg/100m)': 'Rot.err.[deg/100m]'})

df['for_copy'] = df.apply (lambda row: "{:2.2f}".format(row["Transl. Error[%]"])+"/"+"{:2.2f}".format(row["Rot.err.[deg/100m]"]) , axis=1)
df['for_copy_ATE'] = df.apply (lambda row: "{:2.2f}".format(row["RPE(m)"])+"/"+"{:2.2f}".format(row["ATE(m)"]) , axis=1)
df['method'].replace(['baseline-cfear-1'], 'grid-search-cfear-1') #Correct wrong namer
df['method'].replace(
    to_replace=['baseline-cfear-1'],
    value='grid-search-cfear-1',
    inplace=True
)



./data/run_4/grid-search-cfear-2_2022-04-10_1803_eval.csv
./data/run_4/grid-search-cfear-1_2022-04-10_1134_eval.csv
./data/run_4/grid-search-cfear-3_2022-04-12_1057_eval.csv
./data/run_4/grid-search-cfear-1_2022-04-10_1136_eval.csv
./data/run_4/grid-search-cfear-2_2022-04-10_1422_eval.csv
./data/run_4/grid-search-cfear-3_2022-04-14_1154_eval.csv


In [5]:
#################### NOTE, THIS SCRIPT TAKES TIME TO RUN - a few minutes ##############

error_metrics=["Transl. Error[%]", "Rot.err.[deg/100m]", "RPE(m)", "ATE(m)"]
methods=np.array(df["method"].unique())
datasets=np.array(df["dataset"].unique())
print("------------SCV------------")
print("kstrongest")
print(df["k strongest"].unique())

print("z min")
print(df["z min"].unique())

print("resolution r [m]")
print(df["resolution r [m]"].unique())

print("Weight")
print(df["Weight"].unique())

imin=df['Transl. Error[%]'].argmin()
print(datasets)
print(methods)

#print(df.iloc[imin,:]["Transl. Error[%]"])
for error in error_metrics: # average rotation and translation error over sequence
    df[ (error+' - overall') ] = df.apply (lambda row:  df[ (df["k strongest"]==row["k strongest"]) & (df["z min"]==row["z min"]) & (df["resolution r [m]"]==row["resolution r [m]"]) & (df["Weight"]==row["Weight"]) & (df["method"]==row["method"]) & (df["dataset"]==row["dataset"]) ][error].mean(), axis=1)
    
    #Produce "For copy" - only formating changes
df["mean"] = df.apply (lambda row: "{:2.2f}".format(row["Transl. Error[%] - overall"])+"/"+"{:2.2f}".format(row["Rot.err.[deg/100m] - overall"]) , axis=1)         
df["RPE/ATE-mean"] = df.apply (lambda row: "{:2.2f}".format(100*row["RPE(m) - overall"])+"/"+"{:2.2f}".format(row["ATE(m) - overall"]) , axis=1)         
df["mean SCV"] = df.apply (lambda row: "N/A" , axis=1)         

for dataset_eval in datasets:
    print("-----------------------------------------")
    for method in methods:
        for dataset_training in datasets:
            if dataset_training != dataset_eval:
                df_training=df[ (df["method"]==method) & (df["dataset"]==dataset_training)] # Subset
                df_eval=df[ (df["method"]==method) & (df["dataset"]==dataset_eval)]
                
                #Get training error
                argmin_frame=df_training.iloc[df_training['Transl. Error[%] - overall'].argmin(),:] 
                training_error = str(argmin_frame["mean"]) 
                
                #Get evaluation errror
                mask=(df_eval["k strongest"]==argmin_frame["k strongest"]) & (df_eval["z min"]==argmin_frame["z min"]) & (df_eval["resolution r [m]"]==argmin_frame["resolution r [m]"]) & (df_eval["Weight"]==argmin_frame["Weight"]) 
                evaluation_error=df_eval[mask]["mean"].iloc[0] #Get evaluiat
                evaluation_error_rpe=df_eval[mask]["RPE/ATE-mean"].iloc[0] #Get evaluiat
                eval_mask=(df["method"]==method) & (df["dataset"]==dataset_eval) 
                df.loc[eval_mask,"mean SCV"] = evaluation_error
                df.loc[eval_mask,"mean tailored"] = training_error
                #df_eval["mean SCV"] = df_eval.apply (lambda row: evaluation_error , axis=1)         
                
                #Store evaluation errror
                #df.loc[mask, ["mean SCV"]] = evaluation_error
                #df_training.iloc[train_argmin,:]
                #print("method: "+method+", training dataset: "+dataset_training+", training error="+training_error+", evaluation dataset: "+dataset_eval+"="+evaluation_error)                
                print("method: "+method+", "+dataset_training+", Mean Opti. ="+training_error)
                print("method: "+method+", ,"+dataset_eval+", Mean SCV. ="+evaluation_error)
                print("method: "+method+", ,"+dataset_eval+", RPE/ATE SCV. ="+evaluation_error_rpe)
        
#print(df)
    
df.to_csv('./data/summery_with_SCV.csv', index=False)  

------------SCV------------
kstrongest
[15 10 12 50 30 40]
z min
[60 70 80 50]
resolution r [m]
[2.5  3.5  3.   4.   2.75]
Weight
[0 4]
['oxford' 'mulran']
['grid-search-cfear-2' 'grid-search-cfear-1' 'grid-search-cfear-3']
-----------------------------------------
method: grid-search-cfear-2, mulran, Mean Opti. =1.97/0.66
method: grid-search-cfear-2, ,oxford, Mean SCV. =1.44/0.49
method: grid-search-cfear-2, ,oxford, RPE/ATE SCV. =6.08/47.22
method: grid-search-cfear-1, mulran, Mean Opti. =2.44/0.81
method: grid-search-cfear-1, ,oxford, Mean SCV. =1.79/0.60
method: grid-search-cfear-1, ,oxford, RPE/ATE SCV. =5.52/57.85
method: grid-search-cfear-3, mulran, Mean Opti. =1.59/0.56
method: grid-search-cfear-3, ,oxford, Mean SCV. =1.27/0.39
method: grid-search-cfear-3, ,oxford, RPE/ATE SCV. =5.13/19.52
-----------------------------------------
method: grid-search-cfear-2, oxford, Mean Opti. =1.42/0.50
method: grid-search-cfear-2, ,mulran, Mean SCV. =1.99/0.68
method: grid-search-cfear-2, ,m